<a id="model-selection"></a>

## 4. Sélection de modèles




<a id="logistic-reg"></a>
### 4.1 Régression Logistique

<a id="4.4.1."></a>
#### 4.4.1. Théorie de la régression logistique

Le modèle de régression logistique sert à modéliser une variable d'intérêt de type Bernoulli *Y* à l'aide d'une fonction logistique. Dans un tel cas, la variable *Y* représente la probabilité de succès d'un évènement, c'est-à-dire qu'elle prend une valeur entre 0 et 1. Le lien entre la variable *Y* et les *p* variables explicatives ne peut alors pas être exprimé selon une relation linéaire. Cependant, on peut considérer une relation linéaire entre la fonction logit et les *p* variables explicatives. Dans la régression logistique, les coefficients de régression permettent de représenter l'effet de chaque variable expliative sur la variation de la cote (rapport entre la probabilité de succès et la probabilité d'échec) et ceux-ci peuvent être estimé à l'aide de la méthode du maximum de la vraisemblance.

Tout comme les autres modèles statistiques, la régression logistique possède des points forts et faibles:

Les avantages de la régression logistique sont:
- La facilité de l'implémentation
- Les variables explicatives n'ont pas à être standardizé

Les désavantages de la régression logistique sont:
- Une sélection des variables explicatives nécessaire
- Pas très performant

D'abord, cette méthode est très facile à implémenter, car il n'y a pas beaucoup d'étapes à exécuter et ce n'est pas exigeant en terme de puissance computationnelle. Aussi, les variables explicatives n'ont pas besoin d'être standardizé, donc seulement peu de manipulations pour la transformation de données sont nécessaires pour construire ce modèle. Des désavantages sont également présents. La régression logistique ne peut être performant que si les variables explicatives dont elle utilise sont bien sélectionnées. Des variables qui n'affectent pas du tout la variable de prédiction ou plusieurs varibles qui sont corrélées entre elles peuvent être des bruits pour le modèle et influencer négativement le résultat de la prédiction. De plus, de manière générale, la régression logistique n'est pas un algorithme de classification super performant. D'autres méthodes donnent souvent de meilleurs résultats de prédiction selon le type de problème à résoudre.

#### 4.1.1 Entraînement et prédiction du modèle

L'entraînement et la prédiction du modèle de régression logistique ne sont pas très compliqués. Il suffit de d'abord bien choisir les variables explicatives à évaluer et ensuite à modéliser la variable à prédire selon les variables explicatives choisies à l'aide de la fonction logistique. Il reste par la suite à choisir le seuil qui permet de trier le mieux possible les prédictions réalisées (qui sont des probabilités entre 0 et 1).

##### 4.1.1.1 Sélection des variables explicatives

Ici, nous définissons les variables explicatives pouvant être utilisées pour notre modèle. Comme mentionné dans la section précédente où l'on décrit la théorie du présent modèle, il est impératif de choisir les bonnes variables explicatives afin de réduire le plus possible les problèmes comme le surapprentissage. On ne prend que les variables qui semblent affecter la présence d'une surverse (c'est-à-dire, la position des ouvrages, la date et la quantité de précipitations):

In [ ]:
names_glm = [:TP_LAT, :TP_LNG, :TP_Z, :MONTH, :DAY, :PCP_SUM, :PCP_MAX, :PCP_MAX3, :SURVERSE];

##### 4.1.1.2 Création du modèle

Nous crééons le modèle de régression logistique en utilisant la fonction glm avec la liste de variables explicatives qui y seront évaluées.
La fonction glm permet de construire un modèle de *n* variables explicatives à l'aide de la relation *$log_2(n+1)$*

In [ ]:
train_features = X_train[:, names_glm];
val_form = @formula(SURVERSE ~ TP_LAT + TP_LNG + TP_Z + MONTH + DAY + PCP_SUM + PCP_MAX + PCP_MAX3);

val_model = glm(val_form, train_features, Bernoulli(), LogitLink());

##### 4.1.1.3 Validation du modèle

Nous pouvons enfin faire la validation de notre modèle. On fait d'abord des prédictions basées sur les données que nous avons utilisé pour construire le modèle. Les prédictions calculées sont des valeurs entre 0 et 1 qui correspondent à la probabilité qu'une surverse ait lieue.

In [1]:
val_features = val_set[:, names_glm];
val_labels = val_set[!, :SURVERSE];
val_pred = GLM.predict(val_model, val_features);

UndefVarError: UndefVarError: val_set not defined

Nous utilisons comme seuil, 0.15 pour trier les prédictions calculées précédemment. C'est-à-dire qu les valeurs qui sont supérieures au seuil seront remplacées par 1 (il y a surverse) et celles qui sont inférieures par 0 (il n'y a pas surverse).
Ensuite nous convertissons ces valeurs de prédiction en un tableau de *Int* afin de pouvoir déterminer le F1-score.

In [ ]:
threshold = 0.15;
val_pred[val_pred .>= threshold] .= 1.0;
val_pred[val_pred .< threshold] .= 0.0;
val_pred = convert(Array{Int}, trunc.(val_pred))

r = roc(val_labels, val_pred);
f1score(r)

Nous pouvons ensuite tenter de trouver un seuil qui fonctionne le mieux dans notre contexte en réalisant plusieurs essais, en modifiant légèrement la valeur de seuil à chaque itération. Nous commençons avec un seuil de 0.1 et on cherche la valeur qui donne le meilleur F1-score.

In [ ]:
niter = 10;
batch_score = 0;
batch_threshold = 0;

for i=1:niter
    # Split train and val sets
    r_idx = shuffle(1:size(comb, 1));
    train_ceil = floor(Int, size(r_idx, 1) * 0.8);
    train_set = comb[r_idx[1:train_ceil], :];
    val_set = comb[r_idx[train_ceil+1:size(r_idx, 1)], :];
    
    # Build features and labels
    train_features = train_set[:, names_glm];
    
    # Build model
    val_model = glm(val_form, train_features, Bernoulli(), LogitLink())
    
    # Validate model
    val_features = val_set[:, names_glm];
    val_labels = val_set[!, :SURVERSE];
    
    # Get best threshold
    start_threshold = 0.1;
    max_threshold = 0.15;
    step = 0.0002;
    
    best_threshold = start_threshold;
    score = -1;
    
    # Get best threshold
    for j=start_threshold:step:max_threshold
        val_pred = GLM.predict(val_model, val_features);
        val_pred[val_pred .>= j] .= 1.0;
        val_pred[val_pred .< j] .= 0.0;
        val_pred = convert(Array{Int}, trunc.(val_pred))
        
        r = roc(val_labels, val_pred);
        new_score = f1score(r);
        
        if new_score > score
            score = new_score
            best_threshold = j
        end
    end
    
    batch_score += score;
    batch_threshold += best_threshold;
end

batch_threshold = batch_threshold / niter;
batch_score = batch_score / niter

##### 4.1.1.4 Prédiction finale

Calculer la valeur du threshhold et le F1-score

<a id="4.3."></a>
### 4.3. Arbres de décision et forêt aléatoire [TODO: METTRE A JOUR TABLE DES MATIERES]

<a id="4.3.1."></a>
#### 4.3.1. Théorie des arbres de décision

Le modèle d'arbres de décision est un algorithme d'apprentissage machine très puissant capable d'effectuer des tâches de classification, mais aussi de régression. Il permet de présenter le résultat d'une série de décisions qui ont été effectuées à l'aide de plusieurs variables explicatives. Dans notre cas, c'est la partie classification qui nous intéresse, car nous cherchons à déterminer si une surverse a eu lieu ou pas, selon certaines variables explicatives.

Un arbre de décision est modélisé en prenant d'abord les variables explicatives et en calculant les indices de diversité de Gini de chacune. Ce dernier représente la fréquence que l'issu d'un nouvel élément de l'ensemble soit mal classé si la classification des variables explicatives se fait de manière aléatoire. En effet, on calcule l'indice de diversité de Gini de chaque variable explicative et on remplit l'arbre graduellement à partir de celle qui donne la meilleure valeur (les valeurs se situent entre 0 et 1, 0 étant la meilleure) vers velle qui donne la pire.

Les avantages des arbres de décision sont:
- La facilité de l'implémentation
- L'efficacité pour identifier des variables explicatives significatives 

Les désavantages des arbres de décision sont:
- Une faible précision de prédiction

Le modèle d'arbres de décision est très facile à implémenter, des manipulations mathématiques ne sont pas nécessaire afin de réaliser la modélisation. Les données n'ont pas à être standardizé comme c'était le cas pour des modèles comme la régression logistique. De plus, ce modèle est connu pour être efficace en terme d'identification des variables explicatives qui sont significatives et pour repérer les relations entre plusieurs variables explicatives. En effet, la structure de l'arbre créée dépend des indices de diversité de Gini de chacune donc le lien entre les variables explicatives y est encapsulé. Cependant, comme tout autre modèle, les arbres de décision ont également des désavantages. Le principal inconvénient d'utiliser ces derniers est qu'il ne permet pas de donner des prédictions précises. C'est la raison pour laquelle il est rare qu'on utilise un arbre de décision directement sur un ensemble de données; on fait recours à un modèle qui se base sur celui-ci, comme c'est le cas pour le modèle de forêt aléatoire.

<a id="svm"></a>
### 4.4 Séparateur à vaste marge

<a id="possible-improvements"></a>
### 5. Améliorations possibles